# Demonstration of the use of the rsradia.jupyter function mpi_solve
mpi_solve is a wrapper that is designed to function just like one of Radia's built in relaxation commands (together with the RlxPre command), but with the capability to run in parallel. To do this from a notebook mpi_solve dumps the Radia object it is given to disk, calls mpiexec in a subprocess on the appropriate Radia relaxation command given the arguments passed in and then dumps the result to be loaded back into the notebook.

A basic example is shown below using the HybridUndCenPart function defined in radia_test_function.py

In [1]:
import numpy as np
import radia as rad
from radia_test_function import HybridUndCenPart
from math import *

# Define parameters for undulator and create Radia object

In [2]:
#General Undulator Parameters
per = 14.
nPer = 15 #10 
gap = 4.
gapOffset = 0.
air = 0.05

poleThickPer20 = 3.
poleHeight = 19.5
poleWidth = 30. #40.
lp = [poleWidth, poleThickPer20*per/20., poleHeight]
chPole = 0.1 #If cham_pole > 0, it adds pole tip

npx = 6; npy = 6 #5
np = [npx,npy,[12,0.18]] #Pole Subdivision Params
#np = [[6,1.],[5,1.],[12,0.18]]
#np = [npx,npy,12]
npTip = [npx,npy,1] #Pole Tip Subdivision Params
cp = [1.,0.,1.] #Pole Color

magWidth = 45. #58.
lmy = per/2-lp[1]-2*air
lm = [magWidth,lmy,27]

nm = [3,2,[6,1./3.]] #Magnet Subdivision Params
#nm = [[3,1.],[2,1.],[6,1./3.]]
#nm = [3,2,6]
cm = [0.,1.,1.] #Magnet Color

chMagXZ = 3. #Magnet Chamfer in the XZ plane
chMagYZ = 0.05 #Magnet Chamfer in the YZ plane
chMagYZrat = sqrt(3.) #Magnet Chamfer Ratio: Longitudinal/Vertical

#Pole Material 
#B [G] vs H [G] data from NEOMAX
BvsH_G = [[0.,0],[0.5,5000],[1,10000],[1.5,13000],[2,15000],[3,16500],[4,17400],[6,18500],[8,19250],[10,19800],
          [12,20250],[14,20600],[16,20900],[18,21120],[20,21250],[25,21450],[30,21590],[40,21850],[50,22000],
          [70,22170],[100,22300],[200,22500],[300,22650],[500,23000],[1000,23900],[2000,24900]]
MvsH_T = [[BvsH_G[i][0]*1.e-04, (BvsH_G[i][1]-BvsH_G[i][0])*1.e-04] for i in range(len(BvsH_G))]
mp = rad.MatSatIsoTab(MvsH_T)

#Magnet Material
magBr = 1.67 #Remanent Magnetization
mm = rad.MatLin({0.05, 0.15}, magBr)

grp = HybridUndCenPart(_gap=gap, _gap_ofst=gapOffset, _nper=nPer, _air=air,
                       _lp=lp, _ch_p=chPole, _np=np, _np_tip=npTip, _mp=mp, _cp=cp,
                       _lm=lm, _ch_m_xz=chMagXZ, _ch_m_yz=chMagYZ, _ch_m_yz_r=chMagYZrat, _nm=nm, _mm=mm, _cm=cm)

In [3]:
relaxation_args = [

    0.001,

    1000,

    4,

    "ZeroM->False"

]



### Run wrapper Test

In [4]:
from rsradia.jupyter import mpi_solve
import time

In [5]:
start = time.time()
p_res = mpi_solve(grp, relaxation_args)
print("Time:", time.time() - start)

Solve finished
I am rank: 0
Starting relaxation
Finished relaxation
I am rank: 2
I am rank: 1
I am rank: 3

Time: 57.10527539253235


In [6]:
# Expected: ~0.18608 T
print(rad.Fld(p_res, 'bz', [0,0,0]))

0.18608228105372202


# Compare to serial evaluation in notebook

In [8]:
mp2 = rad.MatSatIsoTab(MvsH_T)
mm2 = rad.MatLin({0.05, 0.15}, magBr)
grp2 = HybridUndCenPart(_gap=gap, _gap_ofst=gapOffset, _nper=nPer, _air=air,
                        _lp=lp, _ch_p=chPole, _np=np, _np_tip=npTip, _mp=mp2, _cp=cp,
                        _lm=lm, _ch_m_xz=chMagXZ, _ch_m_yz=chMagYZ, _ch_m_yz_r=chMagYZrat, _nm=nm, _mm=mm2, _cm=cm)

In [ ]:
start = time.time()
IM2 = rad.RlxPre(grp2)
rad.RlxAuto(IM2, relaxation_args[0], relaxation_args[1], relaxation_args[2], relaxation_args[3])
print("Time:", time.time() - start)
print(rad.Fld(grp2, 'bz', [0,0,0]))

# Visualize Magnet (Requires jupyter_rs_radia)

In [7]:
import ipywidgets
from jupyter_rs_radia import radia_viewer

In [8]:
rv = radia_viewer.RadiaViewer()
rv.add_geometry('Simple Dipole Steerer 1', grp)
rv.display()

RadiaViewer(children=(Viewer(children=(VTK(layout=Layout(margin='auto', min_width='25%', width='50%'), model_d…